In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from kfsims.node import make_simple_nodes
from kfsims.common import init_trajectory

In [2]:
traj = init_trajectory()

In [3]:
n = make_simple_nodes(1)[0]
n()
n.post_rmse(traj.X)

4.283616571261836

In [11]:
pool = make_simple_nodes(10)
[n() for n in pool]
[n.post_rmse(traj.X) for n in pool]

[4.283616571261836,
 3.6085119657530247,
 3.6137137365997134,
 3.174203081988801,
 2.155149858279561,
 2.0203934200448312,
 6.241768224833669,
 3.422291777256467,
 4.202213753579578,
 4.565811466819882]